In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

In [2]:
#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

uav_path = '../../../unloaded/*.wav'
loaded_path = '../../../loaded/*.wav'
none_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

uav_files = glob.glob(uav_path)
loaded_files = glob.glob(loaded_path)
none_files = glob.glob(none_path)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
uav_raw = load(uav_files)
loaded_raw = load(loaded_files)
none_raw = load(none_files)

(21554792,)
(8911343,)
(8847360,)


# Data Processing

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_loaded, y_loaded = mfcc4(loaded_raw, 2)
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
mfcc_none, y_none = mfcc4(none_raw, 0)

print(mfcc_uav.shape, y_uav.shape)
print(mfcc_loaded.shape, y_loaded.shape)
print(mfcc_none.shape, y_none.shape)

(8911343,)
small end: (16, 13)
small end: (16, 5)
(21554792,)
small end: (16, 12)
small end: (16, 4)
(8847360,)
small end: (16, 9)
(5261, 16, 16) (5261,)
(2174, 16, 16) (2174,)
(2159, 16, 16) (2159,)


In [8]:
X = np.concatenate((mfcc_loaded, mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_loaded, y_uav, y_none))
print(X.shape, y.shape)

(9594, 16, 16) (9594,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 3
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(9594, 3)


In [10]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [11]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [12]:
print(X_train.shape, X_test.shape)
print(X_val.shape, y_val.shape)
print(y_train.shape, y_test.shape)

(6140, 16, 16) (1919, 16, 16)
(1535, 16, 16) (1535, 3)
(6140, 3) (1919, 3)


In [13]:
### path selection between jaesung & gunhoo ###
#np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
#np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
#np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
#np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_val_chunk_3d', X_val)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_val_chunk_3d', y_val)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)

In [14]:
### path selection between jaesung & gunhoo ###
#X_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
#X_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
#y_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
#y_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')
X_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
X_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
X_val = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_val_chunk_3d.npy')
y_val = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_val_chunk_3d.npy')
y_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
y_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')

# Experiment 3 - One convolutional layer /w no dropout

##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [15]:
tf.reset_default_graph()

In [16]:
n_mfcc = 16
n_frame = 16
n_classes = 3
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.0002  # 0.005
#0.0002 
training_epochs = 40

# Layer 1 

In [17]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])


conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
#dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)

flat = tf.reshape(pool1, [-1, 16*16*1])

In [18]:
dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
#dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=True)
logits = tf.layers.dense(inputs=dense2, units=3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Y_pred = tf.contrib.layers.fully_connected(logits,n_classes,activation_fn = None)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_val2 = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)

In [21]:
#model_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn'
model_path = '../models/CNN/my_test_model_cnn'
saver = tf.train.Saver()

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import itertools as it

In [23]:
###########################
batch_size = 64
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):
    avg_cost = 0
    val_avg_cost =0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}   # keep_prob 삭제  
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch 
        
    y_pred = []
    y_true = []
    ix = 0
    for i in range(0, y_val.shape[0], batch_size):
        val_feed_dict={X:X_val2[i:i+batch_size,:,:,:]}  # for validation, change the data 
        _y_pred = sess.run(logits, feed_dict=val_feed_dict)
        _y_pred = sess.run(tf.argmax(_y_pred,1))
        _y_true = y_val[i:i+batch_size,:]
        
        _y_true = sess.run(tf.argmax(_y_true,1))
        #if ix>10:
        #    break
        #ix +=1
        #print(_y_pred)
        
        y_pred.extend(_y_pred)
        y_true.extend(_y_true)
        #for v in range(len(_y_pred)):
        #    y_pred.append(_y_pred[v])
        #    y_true.append(_y_true[v])
    print(len(y_pred),end=' ')
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost), 'val = ','%f' %(accuracy_score(y_true, y_pred)) )
saver.save(sess, model_path)

1535 Epoch: 0001 cost =  2.966546786 val =  0.854723
1535 Epoch: 0002 cost =  0.541216516 val =  0.885342
1535 Epoch: 0003 cost =  0.390083621 val =  0.886645
1535 Epoch: 0004 cost =  0.556076139 val =  0.887296
1535 Epoch: 0005 cost =  0.352007994 val =  0.919870
1535 Epoch: 0006 cost =  0.369630555 val =  0.919218
1535 Epoch: 0007 cost =  0.366189083 val =  0.927036
1535 Epoch: 0008 cost =  0.356032163 val =  0.923127
1535 Epoch: 0009 cost =  0.324624063 val =  0.920521
1535 Epoch: 0010 cost =  0.325517133 val =  0.930293
1535 Epoch: 0011 cost =  0.319874562 val =  0.925081
1535 Epoch: 0012 cost =  0.280389845 val =  0.929642
1535 Epoch: 0013 cost =  0.339118465 val =  0.934202
1535 Epoch: 0014 cost =  0.317104301 val =  0.914007
1535 Epoch: 0015 cost =  0.302314874 val =  0.882085
1535 Epoch: 0016 cost =  0.250117374 val =  0.913355
1535 Epoch: 0017 cost =  0.283208243 val =  0.884691
1535 Epoch: 0018 cost =  0.269384928 val =  0.867752
1535 Epoch: 0019 cost =  0.230661257 val =  0.

'../models/CNN/my_test_model_cnn'

In [24]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2}) # keep_prob 삭제  
y_true = sess.run(tf.argmax(y_test,1))

In [25]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.938
Accuracy:  0.9379885356956749
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       433
          1       0.98      0.91      0.94      1041
          2       0.82      0.95      0.88       445

avg / total       0.94      0.94      0.94      1919

[[431   2   0]
 [  0 946  95]
 [  0  22 423]]
